In [2]:
import pandas as pd
import numpy as np
import json

In [ ]:
# nowe książki z adnotacji 
ks_adnotations_df = pd.read_excel('pbl_ks_adnotacje_20-10-2024.xlsx').fillna('')

ks_adnotations_df['adnotations_full'] = ks_adnotations_df['ZA_ADNOTACJE'] + ' ' + ks_adnotations_df['ZA_ADNOTACJE2'] + ' ' + ks_adnotations_df['ZA_ADNOTACJE3']
ks_adnotations_df = ks_adnotations_df.sample(frac=1).reset_index(drop=True)
ks_adnotations_df['group'] = np.arange(len(ks_adnotations_df)) // 5

ks_adnotations_dct = {}
for idx, row in ks_adnotations_df.iterrows():
    ks_adnotations_dct.setdefault(100001 + row['group'], []).append((row['ZA_ZAPIS_ID'], row['adnotations_full'].strip('\n')))
    
with open('ks_adnotations_dct.json', 'w', encoding='utf-8') as jfile:
    json.dump(ks_adnotations_dct, jfile, indent=4, ensure_ascii=False)
    
for key, val in ks_adnotations_dct.items():
    with open(f'txt_files/KS/{key}.txt', 'a', encoding='utf-8') as txt:
        for tup in val:
            txt.writelines(tup[1] + '\n')

In [2]:
# dzielenie zbiorówek
zbiorowki_adnotations_df = pd.read_excel('zbiorowki_27-01-2025.xlsx').fillna('')

zbiorowki_adnotations_df['adnotations_full'] = zbiorowki_adnotations_df['ZA_ADNOTACJE'] + ' ' + zbiorowki_adnotations_df['ZA_ADNOTACJE2'] + ' ' + zbiorowki_adnotations_df['ZA_ADNOTACJE3']
zbiorowki_adnotations_df = zbiorowki_adnotations_df.sample(frac=1).reset_index(drop=True)
zbiorowki_adnotations_df = zbiorowki_adnotations_df[['ZA_ZAPIS_ID','adnotations_full']]
zbiorowki_adnotations_df = zbiorowki_adnotations_df.sample(frac=1).reset_index(drop=True)

c:\Users\Nikodem\Documents\main_venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
zbiorowki_adnotations_df['adnotations_full'] = zbiorowki_adnotations_df['adnotations_full'].str.split('. - ', regex=False)
zbiorowki_adnotations_df = zbiorowki_adnotations_df.explode('adnotations_full', ignore_index=True)

In [3]:
zbiorowki_adnotations_df[zbiorowki_adnotations_df['ZA_ZAPIS_ID'] == 4747]['adnotations_full']

7389    M. Wyka: Makuszyński - poeta [wstęp]. * [Wiers...
Name: adnotations_full, dtype: object

In [3]:
counter = 0
for idx, row in zbiorowki_adnotations_df.iterrows():
    if counter == 600: break
    file_id = row['ZA_ZAPIS_ID']
    adnotation = row['adnotations_full'].replace('\n', ' ')
    with open(f'txt_files/ZBIOROWKI/zapis_id_{file_id}.txt', 'a', encoding='utf-8') as txt:
        txt.writelines(adnotation)
    counter += 1

In [8]:
# load ks_adnotations_dct to get ids of records used for training
with open('ks_adnotations_dct.json', 'r', encoding='utf-8') as jfile:
    ks_adnotations_dct = json.load(jfile)

with open('training_groups.txt', 'r', encoding='utf-8') as txt:
    training_groups = txt.readlines()
    training_groups = [e.strip() for e in training_groups]

ids = [e[0] for k,v in ks_adnotations_dct.items() if k in training_groups for  e in v]

with open('ks_training_ids.json', 'w', encoding='utf-8') as jfile:
    json.dump(ids, jfile, indent=4, ensure_ascii=False)